In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import gaussian_kde

from sklearn.model_selection import train_test_split

# Constants

In [2]:
alpha = 1.0

# Preprocessing

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)
pd.options.display.max_columns = 100

In [4]:
%%time
df = pd.read_csv('data/train_2011_2012_2013.csv', sep=";", nrows=1e10)

<string>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 1min 42s


In [6]:
df.head()

,DATE,DAY_OFF,DAY_DS,WEEK_END,DAY_WE_DS,TPER_TEAM,TPER_HOUR,SPLIT_COD,ACD_COD,ACD_LIB,ASS_SOC_MERE,ASS_DIRECTORSHIP,ASS_ASSIGNMENT,ASS_PARTNER,ASS_POLE,ASS_BEGIN,ASS_END,ASS_COMENT,CSPL_I_STAFFTIME,CSPL_I_AVAILTIME,CSPL_I_ACDTIME,CSPL_I_ACWTIME,CSPL_I_ACWOUTTIME,CSPL_I_ACWINTIME,CSPL_I_AUXOUTTIME,CSPL_I_AUXINTIME,CSPL_I_OTHERTIME,CSPL_ACWINCALLS,CSPL_ACWINTIME,CSPL_AUXINCALLS,CSPL_AUXINTIME,CSPL_ACWOUTCALLS,CSPL_ACWOUTIME,CSPL_ACWOUTOFFCALLS,CSPL_ACWOUTOFFTIME,CSPL_AUXOUTCALLS,CSPL_AUXOUTTIME,CSPL_AUXOUTOFFCALLS,CSPL_AUXOUTOFFTIME,CSPL_INFLOWCALLS,CSPL_ACDCALLS,CSPL_ANSTIME,CSPL_HOLDCALLS,CSPL_HOLDTIME,CSPL_HOLDABNCALLS,CSPL_TRANSFERED,CSPL_CONFERENCE,CSPL_ABNCALLS,CSPL_ABNTIME,CSPL_ABNCALLS1,CSPL_ABNCALLS2,CSPL_ABNCALLS3,CSPL_ABNCALLS4,CSPL_ABNCALLS5,CSPL_ABNCALLS6,CSPL_ABNCALLS7,CSPL_ABNCALLS8,CSPL_ABNCALLS9,CSPL_ABNCALLS10,CSPL_OUTFLOWCALLS,CSPL_OUTFLOWTIME,CSPL_MAXINQUEUE,CSPL_CALLSOFFERED,CSPL_I_RINGTIME,CSPL_RINGTIME,CSPL_RINGCALLS,CSPL_NOANSREDIR,CSPL_MAXSTAFFED,CSPL_ACWOUTADJCALLS,CSPL_AUXOUTADJCALLS,CSPL_DEQUECALLS,CSPL_DEQUETIME,CSPL_DISCCALLS,CSPL_DISCTIME,CSPL_INTRVL,CSPL_INCOMPLETE,CSPL_ACCEPTABLE,CSPL_SERVICELEVEL,CSPL_ACDAUXOUTCALLS,CSPL_SLVLABNS,CSPL_SLVLOUTFLOWS,CSPL_RECEIVED_CALLS,CSPL_ABANDONNED_CALLS,CSPL_CALLS,CSPL_ACWTIME,CSPL_ACDTIME
0,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,855,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,5400,4812,0,0,0,0,0,0,543,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,30,0,0,30,0,0,0,0,0,0,0,0
1,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1587,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,12600,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,30,0,0,15,0,0,0,0,0,0,0,0
2,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1589,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,7200,2632,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,30,0,0,20,0,0,0,0,0,0,0,0
3,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1591,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,10800,832,0,0,0,0,0,0,498,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0
4,2011-04-24 01:30:00.000,0,NaN,1,Dimanche,Nuit,1,1555,1,Entity1 G3RV6,Entity1 France,DSIG,Téléphonie,Téléphonie,SUPPORT,2001-01-01 00:00:00.000,9999-12-31 00:00:00.000,NaN,12600,2632,488,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,10,10,1,0,7,0,0,0,0,0,0,30,0,1,15,0,0,0,1,0,1,0,488


In [5]:
target = "CSPL_RECEIVED_CALLS"

### deleting all columns but very few

In [7]:
df = df[['DATE', target, 'ASS_ASSIGNMENT']]

In [8]:
df.head()

,DATE,CSPL_RECEIVED_CALLS,ASS_ASSIGNMENT
0,2011-04-24 01:30:00.000,0,Téléphonie
1,2011-04-24 01:30:00.000,0,Téléphonie
2,2011-04-24 01:30:00.000,0,Téléphonie
3,2011-04-24 01:30:00.000,0,Téléphonie
4,2011-04-24 01:30:00.000,1,Téléphonie


In [9]:
df.DATE = pd.to_datetime(df.DATE)

In [10]:
import datetime

get the time slot of the day (0 for 00:00, 1 for 00:30, ... 47 for 23:30)

In [11]:
def get_slot(timestamp):
    assert timestamp.minute in [0, 30]
    return 2 * timestamp.hour + (timestamp.minute == 30)

In [12]:
%%time
df['slot'] = df.DATE.apply(get_slot)

Wall time: 30.8 s


In [13]:
%%time
df['dayofweek'] = df.DATE.apply(lambda x: x.dayofweek)

Wall time: 30.7 s


In [17]:
%%time
df['ass_hashed'] = df.ASS_ASSIGNMENT.apply(hash)

Wall time: 1.91 s


In [18]:
df.head()

,DATE,CSPL_RECEIVED_CALLS,ASS_ASSIGNMENT,slot,dayofweek,ass_hashed
0,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
1,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
2,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
3,2011-04-24 01:30:00,0,Téléphonie,3,6,-1730673935
4,2011-04-24 01:30:00,1,Téléphonie,3,6,-1730673935


### simple model : predict constant value per (ASS_ASSIGNMENT, SLOT, DAYOFWEEK) unique tuple

In [19]:
def best_mean(s):
    """The constant that minimizes the empirical loss.
    For numerical stability, we multiply and divide by the max.
    Equal to 1 / alpha * np.log(1 /s.size * np.sum(np.exp(alpha*s)))"""
    s = alpha * s
    m = s.max()
    s = s - m
    return 1 / alpha * (np.log(1/s.size) + m + np.log(np.sum(np.exp(s))))

In [20]:
def get_best_constant(row):
    
    # selecting corresponding time slots in train set
    sub_df = df[(df.ass_hashed == hash(row.ASS_ASSIGNMENT)) & (df.slot == row.slot) & (df.dayofweek == row.dayofweek)]
#     sub_df = sub_df[sub_df.slot == row.slot]
#     sub_df = sub_df[sub_df.dayofweek == row.dayofweek]
    
    if sub_df.empty:
        print("no data for row={}".format(row))
        return 0

    # get the best prediction constant for that loss
    s = sub_df.reset_index().groupby("DATE")[target].sum()
    return best_mean(s)

In [41]:
%%timeit
get_best_constant(df.iloc[10])

10 loops, best of 3: 97.7 ms per loop


# Read submission data

In [22]:
sub = pd.read_csv('data/submission.txt', sep='\t')

In [23]:
sub.DATE = pd.to_datetime(sub.DATE)

In [24]:
# there are no training data for the submission dates!
sub_ix = sub.set_index('DATE').index
df_ix = df.set_index("DATE").index
df_ix.intersection(sub_ix)

DatetimeIndex([], dtype='datetime64[ns]', name=u'DATE', freq=None)

In [36]:
sub.shape

(82909, 5)

In [37]:
sub['dayofweek'] = sub.DATE.apply(lambda x: x.dayofweek)
sub['slot'] = sub.DATE.apply(get_slot)

In [38]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction,dayofweek,slot
0,2012-12-28,CMS,-inf,4,0
1,2012-12-28,Crises,-inf,4,0
2,2012-12-28,Domicile,-inf,4,0
3,2012-12-28,Gestion,-inf,4,0
4,2012-12-28,Gestion - Accueil Telephonique,-inf,4,0


In [39]:
sub.iloc[15]

DATE              2012-12-28 00:00:00
ASS_ASSIGNMENT              Tech. Axa
prediction                       -inf
dayofweek                           4
slot                                0
Name: 15, dtype: object

In [40]:
%%time
get_best_constant(sub.iloc[15])

Wall time: 106 ms


-inf

### cell below takes ~3 hours to run

In [28]:
for i in range(sub.shape[0]):
    if not i % 100:
        print("{} %".format(100 * i / sub.shape[0]))
    sub.loc[i, 'prediction'] = get_best_constant(sub.iloc[i])

0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
0 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
1 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
2 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
3 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
4 %
5 %
5 %
5 %


KeyboardInterrupt: 

In [29]:
sub.prediction.describe()

count    8.290900e+04
mean             -inf
std               NaN
min              -inf
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      0.000000e+00
Name: prediction, dtype: float64

In [ ]:
sub.head()

In [ ]:
real_sub = sub.iloc[:, :3]

In [ ]:
real_sub.head()

In [ ]:
real_sub.DATE = real_sub.DATE.astype(str) + ".000"

In [ ]:
real_sub.to_csv('data/sub3.txt', sep='\t', index=None)

In [ ]:
# real_sub.prediction = real_sub.prediction.astype(np.int64)